In [3]:
from pathlib import Path
import polars as pl
from pprint import pprint
from continuous import components

In [12]:
def choose_by_length(minimum: int|str, maximum:int|str=420000):
    """checks the length of ohlc history of each trading pair, then makes a list of all pairs whos history length falls
    within the stated range"""

    lengths = {'1 month': 8750, '2 months': 17500, '3 months': 26250, '6 months': 52500,
               '1 year': 105000, '2 years': 210000, '3 years': 315000, '4 years': 420000}

    if isinstance(minimum, str):
        minimum = lengths[minimum]
    if isinstance(maximum, str):
        maximum = lengths[maximum]

    info = {}
    data_path = Path("/home/ross/coding/modular_trader/bin_ohlc_5m")
    for pair_path in list(data_path.glob('*')):
        df = pl.read_parquet(pair_path)
        info[pair_path.stem] = len(df)
    
    print(min(info.values()))

    return [p for p, v in info.items() if minimum < v <= maximum]

In [14]:
choose_by_length(0, 100)

56


['BOMEUSDT']